# Aulas Práticas - EAE

In [27]:
import pandas as pd
import seaborn as sns
import numpy as np
import statsmodels.api as sm
from statsmodels.sandbox.regression.gmm import IV2SLS
from math import log


## ***Aula 2022/04/10***

***Exemplo 15.7 Wooldrige***

```r 
library(AER); library(lmtest)

#restringir salarios observados
oursample <- subset(MROZ, !is.na(wage))

# primeiro estagio:forma reduzida
stage1 <- lm(educ ~exper+I(exper^2)+motheduc+fatheduc, data=oursample)

#segundo estagio
stage2 <- lm(log(wage)~educ+exper+I(exper^2)+resid(stage1), data=oursample)

#resultados incluindo testes t
coeftest(stage2)
```

***Exemplo Teste de Hausman***

```r
library(AER)

#restringir salarios observados
oursample <- subset(MROZ, !is.na(wage))

#estimar 2SLS
ivreg1<-ivreg(log(wage)~educ+exper+I(exper^2)|motheduc+fatheduc+exper+I(exper^2), data=oursample)

#teste Hausmann
summary(ivreg1, diagnostics=TRUE)
```

In [37]:
from numpy import append, apply_along_axis


df = pd.read_stata('dados/MROZ.DTA')
df = df.loc[df['wage'].notna()]
df['exper**2'] = df['exper'] ** 2

vars_x1 = sm.add_constant(df[['exper', 'exper**2', 'motheduc', 'fatheduc']])
y1 = df['educ']
stage1 = sm.OLS(y1, vars_x1, missing='drop').fit()

vars_x2 = sm.add_constant(df[['educ', 'exper', 'exper**2']])
vars_x2['resid_stage1'] = stage1.resid                                                               
y2 = df['wage'].apply(log)
stage2 = sm.OLS(y2, vars_x2, missing='drop').fit()
display(print(stage2.summary()))

                            OLS Regression Results                            
Dep. Variable:                   wage   R-squared:                       0.154
Model:                            OLS   Adj. R-squared:                  0.146
Method:                 Least Squares   F-statistic:                     19.31
Date:                Tue, 04 Oct 2022   Prob (F-statistic):           1.33e-14
Time:                        10:42:19   Log-Likelihood:                -432.22
No. Observations:                 428   AIC:                             874.4
Df Residuals:                     423   BIC:                             894.7
Df Model:                           4                                         
Covariance Type:            nonrobust                                         
                   coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------
const            0.1519      0.398      0.382   

None

In [45]:
exog = sm.add_constant(df[['exper', 'exper**2']])
instruments = sm.add_constant(df[['exper', 'exper**2', 'motheduc', 'fatheduc']])
y = df['wage'].apply(log)

model = IV2SLS(endog=y, exog=exog, instrument=instruments).fit()
print(model.summary())

                          IV2SLS Regression Results                           
Dep. Variable:                   wage   R-squared:                       0.031
Model:                         IV2SLS   Adj. R-squared:                  0.027
Method:                     Two Stage   F-statistic:                     6.864
                        Least Squares   Prob (F-statistic):            0.00116
Date:                Tue, 04 Oct 2022                                         
Time:                        19:58:09                                         
No. Observations:                 428                                         
Df Residuals:                     425                                         
Df Model:                           2                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.9737      0.068     14.352      0.0

In [46]:
print(model.spec_hausman())

(-1.511757977540514e-13, 1.0, 3)


In [43]:
round(0.1267127193620064, 3)

0.127

In [ ]:
b_mqo = stage1.params
b_vi = stage2.params

A = (b_vi - b_mqo).T
B = np.linalg.inv(stage2.cov_params() - stage1.cov_params())
C = (b_vi - b_mqo)
# H = A * B * C

In [33]:
C

const          -8.472314
exper           0.002634
exper**2       -0.000783
fatheduc       -0.170273
motheduc       -0.155858
resid_stage1         NaN
dtype: float64